In [17]:
import importlib
import core_images
import os

importlib.reload(core_images)

#get list of all images in directories
image_dirs = ["data/Core Images/3-03- 11/"]#,"data/Core Images/21-20a- 1/","data/Core Images/42-29- 6/","data/Core Images/206-01- 2/"]
images = []
for dir in image_dirs:
    files = os.listdir(dir)
    dir_images = [os.path.join(dir, f) for f in files]
    images += dir_images

core_x = 730
core_x_size = 6240
core_x_cm = 955
core_y_size = 865
core_x = (core_x,core_x + core_x_size)
core_y= (
    (1470,1470 + core_y_size),
    (2550,2550 + core_y_size),
    (3615,3615 + core_y_size)
)

slice = 100
step = slice/2

cores = core_images.CoreImages(images, core_x, core_y, core_x_cm, slice, step)

cores.slice_cores("data/cores/", "data/slices/", verbose = 1)

Processing core photo: data/Core Images/3-03- 11/S00066841.jpg
Processing core photo: data/Core Images/3-03- 11/S00066846.jpg
Processing core photo: data/Core Images/3-03- 11/S00066844.jpg
Processing core photo: data/Core Images/3-03- 11/S00066847.jpg
Processing core photo: data/Core Images/3-03- 11/S00066845.jpg
Processing core photo: data/Core Images/3-03- 11/S00066848.jpg
Processing core photo: data/Core Images/3-03- 11/S00066842.jpg
Processing core photo: data/Core Images/3-03- 11/S00066843.jpg
